In [1]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np

In [37]:
brands=[]
artics=[]
categories=[]
temporary_links = []
primary_links = []
product_links = []
titles = []
prices = []

url = 'https://www.ombratools.ru/catalog/'
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')
for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")}):
    temporary_links.append(link.get('href'))


In [38]:
temporary_links

['/catalog/',
 '/default/catalog/group/type/new/',
 '/default/catalog/group-release/',
 '/catalog/',
 '/default/catalog/group/type/new/',
 '/default/catalog/group/type/top/',
 '/catalog/10001/klyuchi-gaechniye/',
 '/catalog/20000000/naboriy-instrumenta/',
 '/catalog/20000002/golovki-tortseviye/',
 '/catalog/20000003/privodiy-vorotki-treschotki/',
 '/catalog/20000005/sharnirno-gubtseviyy-instrument/',
 '/catalog/20000006/instrument-dlya-rabotiy-s-vnutrennim-profilem/',
 '/catalog/20000007/instrument-spetsialnogo-naznacheniya/',
 '/catalog/20000008/pnevmaticheskiy-instrument/',
 '/catalog/20000009/proizvodstvennaya-mebel/',
 '/catalog/20000010/garajnoe-oborudovanie/',
 '/catalog/2000003977/vspomogatelniyy-instrument/']

In [39]:
for link in np.unique(temporary_links):
    if  len(re.split(r'(\W+)', link)) >= 6 and any(map(str.isdigit, re.split(r'(\W+)', link)[4])):
        primary_links.append(link)

In [40]:

        
for primary_link in primary_links:
    url = 'https://www.ombratools.ru' + primary_link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    temporary_links = []
    for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")}):
        temporary_links.append(link.get('href'))
    for link in np.unique(temporary_links):
        if link not in primary_links and len(re.split(r'(\W+)', link)) >= 6 and any(map(str.isdigit, re.split(r'(\W+)', link)[4])):
            primary_links.append(link)
    for link in soup.findAll('a', attrs={'href': re.compile("/product/")}):
        if link not in product_links:
            product_links.append(link.get('href'))
product_links = np.unique(product_links)

In [41]:
len(product_links)

826

In [42]:


for product_link in product_links:
    url = 'https://www.ombratools.ru' + product_link
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    
    
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    
    title = soup.find_all('title')[0].text
    titles.append(title)
    
    tags = soup.find_all('div')
    for tag in tags:
        if 'р.' in tag.get_text():
            price = tag.get_text()
    price = "".join(price.replace("р.","").split())
    prices.append(price)
    
    numb=str(soup.find_all('h1')).find('артикул')
    artic=str(soup.find_all('h1'))[numb+9:-6]
    artics.append(artic)
    brands.append('Ombratools')
    n=soup.find_all('a')
    for i in range(len(n)):
        if n[i].text=='перейти к сравнению продуктов':
            category=n[i-1].text
    categories.append(category)


    
product_links =[ 'https://www.ombratools.ru%s'%x for x in product_links ]

table = pd.DataFrame(list(zip(titles, brands,artics, prices,categories, product_links)), columns =['Наименование товара', 'Бренд','Артикул','Цена, р.', 'Категория','Ссылка']) 

table.to_excel('ombratools.xlsx')

In [43]:
table.to_excel('ombratool2s.xlsx')